<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/master/recipes/plugins_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to develop a Kili Webhook - example

## Context

This notebook is an end-to-end example that you can follow to: create a project, register a first webhook and activate it on this project for the corresponding events in Kili. For more info on available events, please refer to [the doc](https://python-sdk-docs.kili-technology.com/2.129/sdk/plugins/)

Webhooks are really similar to the plugins, except they are self hosted, and will require a webservice deployed on your premise, callable by Kili (You can implement a header-based security).
If you are a Europe SaaS user, plugins & webhooks are available for paying-customers in beta for now. 
If you are a US/ On Premise customer, only webhooks are available as of version 1.128.0.

Webhook allow you to easily access your custom code, manage the CI/CD of the version of the plugin deployed, & easily integrate with your own stack.
If you are looking for a more of-the-shelf capability, you can have a look at `plugin_development` notebook

**NB: The webhook capabilities of Kili are under active development, and compatible with version 2.128.0 and later of Kili. Don't hesitate to reach out via Github or the Kili support to provide feedback.**


## Step 1: Instantiate Kili

In [ ]:
!pip install kili

In [ ]:
from kili.client import Kili
import os

kili = Kili()

## Step 2: Create the project

First, we need to create a new project. In our example, we will use an `IMAGE` type project with the following `jsonInterace`:

In [ ]:
json_interface = {
    "jobs": {
        "JOB_0": {
            "content": {
                "categories": {
                    "OBJECT_A": {
                        "children": [],
                        "name": "Object A",
                        "color": "#733AFB",
                        "id": "category1",
                    },
                    "OBJECT_B": {
                        "children": [],
                        "name": "Object B",
                        "color": "#3CD876",
                        "id": "category2",
                    },
                },
                "input": "radio",
            },
            "instruction": "Categories",
            "isChild": False,
            "tools": ["rectangle"],
            "mlTask": "OBJECT_DETECTION",
            "models": {},
            "isVisible": True,
            "required": 1,
            "isNew": False,
        }
    }
}

In [ ]:
title = "Webhooks test project"
description = "My first project with a webhook"
input_type = "IMAGE"

project = kili.create_project(
    title=title, description=description, input_type=input_type, json_interface=json_interface
)
project_id = project["id"]

print(f"Created project {project_id}")

Upload an asset:

In [ ]:
content_array = ["https://storage.googleapis.com/label-public-staging/car/car_1.jpg"]
names_array = ["landscape2"]

kili.append_many_to_dataset(
    project_id=project_id,
    content_array=content_array,
    external_id_array=names_array,
    disable_tqdm=True,
)

asset_id = list(kili.assets(project_id=project_id, fields=["id"], disable_tqdm=True))[0]["id"]

This project has one job of bounding box creation with two categories.

With our plugin, we want to make sure that the labelers don't create more than one bounding box of category A.

To iterate on the plugin code, you can refer to the plugins_development.ipynb notebook.

## Step 3: Write & host the webhook

The webhook rely on the same handlers provided by the plugins. For maximum compatibility, we encourage you to define it with the same base class. Below is an example with FastAPI webservice.

```python
# file plugin.py
from kili.plugins import PluginCore
from typing import Dict

def check_rules_on_label(label: Dict):
    #custom methods 
    print('Custom method - checking number of bboxes')

    counter = 0
    for annotation in label['jsonResponse']["JOB_0"]["annotations"]:
        if annotation["categories"][0]["name"] == "OBJECT_A":
            counter += 1

    if counter == 0:
        return []
    return [f'There are too many BBox ({counter}) - Only 1 BBox of Object A accepted']

class PluginHandler(PluginCore):
    """
    Custom plugin instance
    """

    def on_submit(self, label: Dict, asset_id: str) -> None:
        """
        Dedicated handler for Submit action 
        """
        self.logger.info("On submit called")

        issues_array = check_rules_on_label(label)

        project_id = self.project_id

        if len(issues_array) > 0:
            print("Creating an issue...")

            for i, _ in enumerate(issues_array):

                self.kili.append_to_issues(
                    label_id=label['id'],
                    project_id=project_id,
                    text=issues_array[i],
                )

            print("Issue created!")

            self.kili.send_back_to_queue(asset_ids=[asset_id])

```

You will need to deploy this on your premise for this to work. Easy solutions are [FastAPI](https://fastapi.tiangolo.com/), with a few lines of codes.
For this demo, we will use https://webhook.site that will allow us to explore the payload of the calls.

```python
# file main.py
from fastapi import FastAPI
from .plugin import PluginHandler

app = FastAPI()
kili = Kili()

@app.post("/")
def main(raw_payload):
    event_type = raw_payload.get('eventType')
    project_id = raw_payload.get('logPayload').get('projectId')

    if (not project_id):
        print('Invalid projectId')
        return
    
    plugin = PluginHandler(kili, project_id)

    if (not event_type):
        print('Invalid event')
        return

    payload = raw_payload.get('payload')
    label = payload.get('label')
    asset_id = payload.get('asset_id')
    
    if (event_type == 'onSubmit'):
        plugin.on_submit(label, asset_id)

    if (event_type == 'onReview'):
        plugin.on_review(label, asset_id)

```

## Step 4: Register & activate the webhook

In [ ]:
from kili.exceptions import GraphQLError
import requests


# we get a new webhook listener
res = requests.post("https://webhook.site/token")
uuid = res.json()["uuid"]
webhook_url_from_browser = f"https://webhook.site/#!/{uuid}"

webhook_name = "Webhook bbox count"
webhook_url = f"https://webhook.site/{uuid}"
print(webhook_url_from_browser)
webhook_security_header = "custom header"

try:
    kili.create_webhook(
        plugin_name=webhook_name, webhook_url=webhook_url, header=webhook_security_header
    )
except GraphQLError as error:
    print(str(error))

In [ ]:
kili.activate_plugin_on_project(plugin_name=webhook_name, project_id=project_id)

**Note**: Similar to plugins, you have access to the methods `kili.update_webhook` & `kili.deactivate_plugin_on_project` for iterations on your code.

## Step 5: Webhook in action

After that, you can test it by labelling in the Kili interface or just by uploading the following label.

When you add the label that contains errors, you will see a new issue automatically created in the Kili app, if you have deployed the webhook.
Else, you can visit the webhook site to check incoming events.

In [ ]:
json_response = {
    "JOB_0": {
        "annotations": [
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.15, "y": 0.84},
                            {"x": 0.15, "y": 0.31},
                            {"x": 0.82, "y": 0.31},
                            {"x": 0.82, "y": 0.84},
                        ]
                    }
                ],
                "categories": [{"name": "OBJECT_A"}],
                "children": {},
                "mid": "20221124161451411-13314",
                "type": "rectangle",
            },
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.79, "y": 0.20},
                            {"x": 0.79, "y": 0.13},
                            {"x": 0.91, "y": 0.13},
                            {"x": 0.91, "y": 0.20},
                        ]
                    }
                ],
                "categories": [{"name": "OBJECT_A"}],
                "children": {},
                "mid": "20221124161456406-47055",
                "type": "rectangle",
            },
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.87, "y": 0.36},
                            {"x": 0.87, "y": 0.27},
                            {"x": 0.99, "y": 0.27},
                            {"x": 0.99, "y": 0.36},
                        ]
                    }
                ],
                "categories": [{"name": "OBJECT_A"}],
                "children": {},
                "mid": "20221124161459298-45160",
                "type": "rectangle",
            },
        ]
    }
}

In [ ]:
kili.append_labels(
    json_response_array=[json_response], asset_id_array=[asset_id], label_type="DEFAULT"
)

If you used & hosted the base webhook provided, the webhook should:

 - Create an issue with information that three bboxes were found, instead of one
 - Send the asset back to the labeling queue (status `ONGOING`)

If you haven't just yet, you can still visit the address here : 

In [ ]:
print(f"Go to my webhook: {webhook_url_from_browser}")

Woah! Amazing! Well done :) 🚀

Let's test now to post a proper label, this one for example:

In [ ]:
json_response = {
    "JOB_0": {
        "annotations": [
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.15, "y": 0.84},
                            {"x": 0.15, "y": 0.31},
                            {"x": 0.82, "y": 0.31},
                            {"x": 0.82, "y": 0.84},
                        ]
                    }
                ],
                "categories": [{"name": "OBJECT_A"}],
                "children": {},
                "mid": "20221124161451411-13314",
                "type": "rectangle",
            }
        ]
    }
}
kili.append_labels(
    json_response_array=[json_response], asset_id_array=[asset_id], label_type="DEFAULT"
)

print(f"Go to my webhook: {webhook_url_from_browser}")

The status of your asset should have now changed to `LABELED`. In this webhook, previous issues remain but you can solve them through the API as well.

Well done! You can now iterate on the script. To learn how to avoid latency when building and deploying your plugin, refer to the *plugins_development.ipynb* tutorial.